In [53]:
import os
import json
import instructor
from openai import OpenAI
from datetime import datetime, timedelta
from dotenv import load_dotenv
from core.messages_service import MessagesService
from core.json_service import dump_json_data

load_dotenv()
client = instructor.patch(OpenAI(api_key=os.getenv("OPENAI_API_KEY")))


In [54]:
# Initialize conversation history
messages_service = MessagesService()
openai_model = "gpt-3.5-turbo"
user_interest_enum = None

# Initial system message
system_prompt = """You are an assistant that helps users find sushi restaurants and parking spots in Munich. 
When a user expresses interest in either sushi, parking, or both:
1. Use the set_user_interest function to record their preference
2. Provide relevant parking or restaurant information
3. ALWAYS respond with a natural language message summarizing the information for the user
"""

messages_service.add_system_message(system_prompt)

user_prompt = "I want to park my car"
# user_prompt = input("eg. I am looking for a Sushi Restaurant in Munich.")
messages_service.add_user_message(user_prompt)

messages_service.print_messages()

{'role': 'system', 'content': 'You are an assistant that helps users find sushi restaurants and parking spots in Munich. \nWhen a user expresses interest in either sushi, parking, or both:\n1. Use the set_user_interest function to record their preference\n2. Provide relevant parking or restaurant information\n3. ALWAYS respond with a natural language message summarizing the information for the user\n'}
{'role': 'user', 'content': 'I want to park my car'}


In [55]:
from tools.user_interest_tools import UserInterest

# Get user interest
intent : UserInterest = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=UserInterest,
)

messages_service.add_assistant_message(intent.assistant_message)
messages_service.add_function_message(UserInterest.__name__, intent.interest)

user_interest_enum = intent.interest

messages_service.print_messages()

# from services.user_interest_service import get_user_interest
# chosen_function = eval(message.tool_calls[0].function.name)
# user_interest = chosen_function(interest)

In [51]:
user_prompt = "Where can I park my car?"
messages_service.add_user_message(user_prompt)

In [52]:
from enums.user_interest_enum import UserInterestEnum
from services.parking_intent_service import ParkingIntentService
from tools.parking_tools import UserParkingIntent
# from tools.sushi_tools import UserSushiIntent

intent_service = ParkingIntentService()
response_model = UserParkingIntent # TODO: add sushi response model

if user_interest_enum == UserInterestEnum.PARKING:
    response_model = UserParkingIntent
elif user_interest_enum == UserInterestEnum.SUSHI:
    pass
    # response_model = UserSushiIntent
else:
    pass
    # response_model = UserParkingIntent + UserSushiIntent

intent = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    response_model=response_model,
)
print(intent)

messages_service.add_assistant_message(intent)
# messages_service.add_function_message(response_model.__name__, intent)

result = intent_service.handle_intent(intent)
print(result)
messages_service.add_assistant_message(result.recommendation)
messages_service.add_function_message(response_model.__name__, result.get("data"))

messages_service.print_messages()



# assistant_message = arguments.get("assistant_message")
# if assistant_message:
#     messages_service.add_assistant_message(assistant_message)
# messages_service.print_last_message()

# function_call_service = FunctionCallService(messages_service)


# message = completion.choices[0].message
# print(message)


# chosen_function = eval(message.tool_calls[0].function.name)
# user_interest = chosen_function()


# user_interest_json = dump_json_data(user_interest)
# print(user_interest_json)

# messages_service.add_function_message(message.tool_calls[0].function.name, user_interest_json)
# messages_service.print_last_message()



# feedback_completion = client.chat.completions.create(
#     model=openai_model,
#     messages=messages_service.get_messages(),
#     tools=tools,
# )

# message = feedback_completion.choices[0].message
# print(message)



query_type='list_all' specific_spot_name=None duration_hours=None height_requirement=None location=None needs_disabled_access=None payment_method_preference=None response_detail='brief'


AttributeError: 'dict' object has no attribute 'recommendation'

In [47]:
# reset jupiter kernel
%reset -f